In [67]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import mixture
from sklearn.neighbors.kde import KernelDensity
from sklearn.cluster import KMeans
from sklearn import svm
'''
Eight  features  (temperature,  dew  point,
sea-level  pressure,  visibility,  average  wind  speed,  max  sus-
tained wind speed, and minimum and maximum temperature)
are  used  to  determine  whether  each  day  experienced  rain  or
no rain.
'''
n_samples = 30#300
n_dimensions = 3
# generate random sample
np.random.seed(0)

#Test 1 with single distribution
randd = np.random.randn(n_samples, n_dimensions)

'''
#Test 2 with two different distributions
# generate spherical data centered on (20, 20)
shifted_gaussian = np.random.randn(n_samples, 2) + np.array([20, 20])

# generate zero centered stretched Gaussian data
C = np.array([[0., -0.7], [3.5, .7]])
stretched_gaussian = np.dot(np.random.randn(n_samples, 2), C)

# concatenate the two datasets into the final training set
#X_train = np.vstack([shifted_gaussian, stretched_gaussian])
'''

X_train = randd

label1 = 0
label2 = 1
classes=[label1, label2]
numClasses = len(classes)


#K-means
kmeans = KMeans(n_clusters=numClasses).fit(X_train)
clusters = kmeans.labels_
#predictedClusters = kmeans.predict([[0, 0, 0], [4, 4, 5]]) #predciting

#Slicing instances according to their inferred clusters
trainClasses = {}
for c in range(numClasses):
    trainClasses[classes[c]]=[X_train[i] for i in range(len(clusters)) if clusters[i] == c]
    

# fit a Gaussian Mixture Model with n components
gmm = {}
for c, points in trainClasses.items():
    clf = mixture.GaussianMixture(n_components=6, covariance_type='full')
    clf.fit(points)
    gmm[c] = clf.score_samples(points)

#Kernel density estimation
kde={}
for c, points in trainClasses.items():
    kernel = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(points)
    kde[c] = kernel.score_samples(points)

print("Original data: ",X_train, "\n")
print("K-Means clusters: ", clusters, "\n")
print("Points from Cluster", label1, ": ", trainClasses[label1], "\n")
print("Points from Cluster", label2, ": ", trainClasses[label2], "\n")
print("PDF for class", label1, "using GMM: ", gmm[label1], "\n")
print("PDF for class", label2, "using GMM: ", gmm[label2], "\n")
print("PDF for class", label1, "using KDE: ", kde[label1], "\n")
print("PDF for class", label2, "using KDE: ", kde[label2], "\n")

#Comparing the results of entire method above with SVM
X = X_train
y = kmeans.labels_ #putting kmeans labels as a response from specialist (yes, i'm lazy)
clf = svm.SVC()
clf.fit(X, y)
svm.SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


print("Sample predicted by SVM: ", clf.predict([[0, 0, 0], [4, 4, 5]]))

Original data:  [[ 1.76405235  0.40015721  0.97873798]
 [ 2.2408932   1.86755799 -0.97727788]
 [ 0.95008842 -0.15135721 -0.10321885]
 [ 0.4105985   0.14404357  1.45427351]
 [ 0.76103773  0.12167502  0.44386323]
 [ 0.33367433  1.49407907 -0.20515826]
 [ 0.3130677  -0.85409574 -2.55298982]
 [ 0.6536186   0.8644362  -0.74216502]
 [ 2.26975462 -1.45436567  0.04575852]
 [-0.18718385  1.53277921  1.46935877]
 [ 0.15494743  0.37816252 -0.88778575]
 [-1.98079647 -0.34791215  0.15634897]
 [ 1.23029068  1.20237985 -0.38732682]
 [-0.30230275 -1.04855297 -1.42001794]
 [-1.70627019  1.9507754  -0.50965218]
 [-0.4380743  -1.25279536  0.77749036]
 [-1.61389785 -0.21274028 -0.89546656]
 [ 0.3869025  -0.51080514 -1.18063218]
 [-0.02818223  0.42833187  0.06651722]
 [ 0.3024719  -0.63432209 -0.36274117]
 [-0.67246045 -0.35955316 -0.81314628]
 [-1.7262826   0.17742614 -0.40178094]
 [-1.63019835  0.46278226 -0.90729836]
 [ 0.0519454   0.72909056  0.12898291]
 [ 1.13940068 -1.23482582  0.40234164]
 [-0.6848

In [37]:
import numpy as np
import pandas as pd
data = pd.read_csv('C:\\Users\\user\\Documents\\Dissertacao\\gsod_2017\\725520-14935-2017.csv',sep = "\t")

#print (data.loc[0][0])